In [1]:
# train a quantum-quantum generative adversarial network on a gaussian probability distribution
import numpy as np
from numpy.random import rand
from numpy.random import randn
from qibo import gates, hamiltonians, models
from matplotlib import pyplot
from scipy.optimize import minimize

In [2]:
# define quantum discriminator cost
def cost_discriminator(params, x, y, samples, nqubits=2, layers=4):
    cost = 0
    circuit = models.Circuit(nqubits)
    index = 0
    for l in range(layers):
        for q in range(nqubits):
            circuit.add(gates.RY(q, theta=0))
            index += 2
        circuit.add(gates.CZ(0,1))
    for q in range(nqubits):
        circuit.add(gates.RY(q, theta=0)) 
        index += 2
    for i in range(samples):
        newparams = rotate_disc(params, x[i][0])
        circuit.set_parameters(newparams)
        # compute cost for real or fake
        if y[i][0] == 1:
            cost += (1+hamiltonian(nqubits).expectation(circuit.execute()).numpy().real)/2
        else:
            cost += (1-hamiltonian(nqubits).expectation(circuit.execute()).numpy().real)/2
    return cost/samples

def rotate_disc(theta, x, layers=4, nqubits=2):
    new_theta = []
    index = 0
    for l in range(layers):
        for q in range(nqubits):
           new_theta.append(theta[index]*x + theta[index+1])
           index += 2
    for q in range(nqubits):
        new_theta.append(theta[index]*x + theta[index+1])
        index += 2
    return new_theta

def rotate_gen(theta, x, layers=4, nqubits=1):
    new_theta = []
    index = 0
    for l in range(layers):
        for q in range(nqubits):
           new_theta.append(theta[index]*x + theta[index+1])
           index += 2
        for q in range(nqubits):
           new_theta.append(theta[index]*x + theta[index+1])
           index += 2
    return new_theta
 
# define the combined generator and discriminator model, for updating the generator
def define_cost_gan(params_gen, params_disc, latent_dim, samples):
    # generate fake samples
    x_fake, y_fake = generate_fake_samples(params_gen, latent_dim, samples)
    # change class
    y_fake = np.ones((samples, 1))
    # evaluate discriminator on fake examples
    accuracy = cost_discriminator(params_disc, x_fake, y_fake, samples)
    return accuracy
 
# generate real samples with class labels
def generate_real_samples(samples, sigma=0.25, mu=0.0):
    # generate samples from the distribution
    s = np.random.normal(mu, sigma, samples)
    # shape array
    X = s.reshape(samples, 1)
    # generate class labels
    y = np.ones((samples, 1))
    return X, y

# define hamiltonian to generate fake samples
def hamiltonian(nqubits=1):
    m0 = (1/nqubits)*hamiltonians.Z(nqubits, numpy=True).matrix
    ham = hamiltonians.Hamiltonian(nqubits, m0)
    return ham
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, samples):
    # generate points in the latent space
    x_input = randn(latent_dim * samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(samples, latent_dim)
    return x_input
 
# use the generator to generate fake examples, with class labels
def generate_fake_samples(params, latent_dim, samples, nqubits=1, layers=4):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, samples)
    # quantum generator circuit
    circuit = models.Circuit(nqubits)
    for l in range(layers):
        for q in range(nqubits):
            circuit.add(gates.RY(q, theta=0))
        for q in range(nqubits):
            circuit.add(gates.RZ(q, theta=0))    
    # generator outputs
    X = []
    for i in range(samples):
        newparams = rotate_gen(params, x_input[i][0])
        circuit.set_parameters(newparams)
        X.append(hamiltonian().expectation(circuit.execute()).numpy().real)
    # shape array
    X = np.array(X).reshape(samples, 1)
    # create class labels
    y = np.zeros((samples, 1))
    return X, y
 
# evaluate the discriminator and plot real and fake distributions
def summarize_performance(epoch, params_gen, params_disc, latent_dim, samples, nbins):
    # prepare real samples
    x_real, y_real = generate_real_samples(samples)
    # evaluate discriminator on real examples
    acc_real = cost_discriminator(params_disc, x_real, y_real, samples)
    acc_real = 1 - acc_real
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(params_gen, latent_dim, samples)
    # evaluate discriminator on fake examples
    acc_fake = cost_discriminator(params_disc, x_fake, y_real, samples)
    # summarize discriminator performance
    print('epoch: ', epoch, 'acc_real: ', acc_real, 'acc_fake: ', acc_fake)
    # histogram plot real and fake data points
    pyplot.hist(x_real, np.linspace(-1.0, 1.0, nbins+1), color='red', label='real', alpha=0.5)
    pyplot.hist(x_fake, np.linspace(-1.0, 1.0, nbins+1), color='blue', label='fake', alpha=0.5)
    pyplot.legend()
    pyplot.show()
 
# train the generator and discriminator
def train(latent_dim, n_epochs=50000, samples=1000, nbins=20, n_eval=1):
    # determine half the size of one batch, for updating the discriminator
    half_samples = int(samples / 2)
    initial_params_gen = np.random.uniform(0, 2*np.pi, 16)
    initial_params_disc = np.random.uniform(0, 2*np.pi, 20)
    # prepare real samples
    x_real, y_real = generate_real_samples(half_samples)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(initial_params_gen, latent_dim, half_samples)
    # stack real and fake
    x = np.vstack((x_real,x_fake))
    y = np.vstack((y_real,y_fake))
    # update discriminator
    train_discriminator = minimize(lambda p: cost_discriminator(p, x, y, samples), initial_params_disc,
                          method='Powell', options={'maxiter': 60, 'maxfev': 60})
    # update generator
    train_generator = minimize(lambda p: define_cost_gan(p, train_discriminator.x, latent_dim, samples), initial_params_gen,
                          method='Powell', options={'maxiter': 48, 'maxfev': 48})
    summarize_performance(1, train_generator.x, train_discriminator.x, latent_dim, samples, nbins)    
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_samples)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(train_generator.x, latent_dim, half_samples)
        # stack real and fake
        x = np.vstack((x_real,x_fake))
        y = np.vstack((y_real,y_fake))
        # update discriminator
        train_discriminator = minimize(lambda p: cost_discriminator(p, x, y, samples), train_discriminator.x,
                          method='Powell', options={'maxiter': 60, 'maxfev': 60})
        # update generator
        train_generator = minimize(lambda p: define_cost_gan(p, train_discriminator.x, latent_dim, samples), train_generator.x,
                          method='Powell', options={'maxiter': 48, 'maxfev': 48}) 
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            summarize_performance(i+2, train_generator.x, train_discriminator.x, latent_dim, samples, nbins)

In [ ]:
# size of the latent space
latent_dim = 1
# train model
train(latent_dim)